In [1]:
import pandas as pd 
import numpy as np
from faKy import values_by_label

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/sandrobarreshamers/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
df = pd.read_table('https://raw.githubusercontent.com/tfs4/liar_dataset/master/train.tsv')
df.columns =['json_id', 'claim', 'object', 'topic', 'speaker', 'job_title (string)','state_info (string)','party_affiliation (string)','barely_true_counts (float32)','false_counts (float32)','half_true_counts (float32)','	mostly_true_counts (float32)','	pants_on_fire_counts (float32)','context (string)']
df2 = df[['json_id', 'claim','object']].copy()

In [3]:
df_test = df2.head(10)
print(df_test)

      json_id        claim                                             object
0  10540.json    half-true  When did the decline of coal start? It started...
1    324.json  mostly-true  Hillary Clinton agrees with John McCain "by vo...
2   1123.json        false  Health care reform legislation is likely to ma...
3   9028.json    half-true  The economic turnaround started at the end of ...
4  12465.json         true  The Chicago Bears have had more starting quart...
5   2342.json  barely-true  Jim Dunnam has not lived in the district he re...
6    153.json    half-true  I'm the only person on this stage who has work...
7   5602.json    half-true  However, it took $19.5 million in Oregon Lotte...
8   9741.json  mostly-true  Says GOP primary opponents Glenn Grothman and ...
9   7115.json  mostly-true  For the first time in history, the share of th...


In [4]:
from faKy import process_text_complexity

df_test['compressed_size'] = df_test['object'].apply(process_text_complexity)


/var/folders/0q/8tgppr255xx8v40x26xhhmhr0000gn/T/ipykernel_85051/1321116620.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['compressed_size'] = df_test['object'].apply(process_text_complexity)


In [5]:
from faKy import process_text_readability
df_test['readability'] = df_test['object'].apply(process_text_readability)
df_test.head()

/var/folders/0q/8tgppr255xx8v40x26xhhmhr0000gn/T/ipykernel_85051/1694183765.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['readability'] = df_test['object'].apply(process_text_readability)


,json_id,claim,object,compressed_size,readability
0,10540.json,half-true,When did the decline of coal start? It started...,11445,71.815000
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",9091,71.781579
2,1123.json,false,Health care reform legislation is likely to ma...,5511,53.655000
3,9028.json,half-true,The economic turnaround started at the end of ...,4709,61.325000
4,12465.json,true,The Chicago Bears have had more starting quart...,12204,54.096667


In [6]:
from faKy import process_text_vader
df_test[['vader_neg', 'vader_neu', 'vader_pos', 'vader_compound']] = df_test['object'].apply(lambda x: pd.Series(process_text_vader(x)))



/var/folders/0q/8tgppr255xx8v40x26xhhmhr0000gn/T/ipykernel_85051/4119106244.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[['vader_neg', 'vader_neu', 'vader_pos', 'vader_compound']] = df_test['object'].apply(lambda x: pd.Series(process_text_vader(x)))
/var/folders/0q/8tgppr255xx8v40x26xhhmhr0000gn/T/ipykernel_85051/4119106244.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[['vader_neg', 'vader_neu', 'vader_pos', 'vader_compound']] = df_test['object'].apply(lambda x: pd.Series(proces

In [7]:
df_test.head()

,json_id,claim,object,compressed_size,readability,vader_neg,vader_neu,vader_pos,vader_compound
0,10540.json,half-true,When did the decline of coal start? It started...,11445,71.815000,0.000,0.902,0.098,0.3612
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",9091,71.781579,0.107,0.687,0.206,0.3182
2,1123.json,false,Health care reform legislation is likely to ma...,5511,53.655000,0.000,0.606,0.394,0.7579
3,9028.json,half-true,The economic turnaround started at the end of ...,4709,61.325000,0.000,1.000,0.000,0.0000
4,12465.json,true,The Chicago Bears have had more starting quart...,12204,54.096667,0.119,0.828,0.053,-0.4601


In [8]:
from faKy import count_named_entities,count_ner_labels, create_input_vector_NER, ner_labels

df_test['tot_ner_count'] = df_test['object'].apply(count_named_entities)
df_test['ner_counts'] = df_test['object'].apply(count_ner_labels)
df_test['input_vector_ner'] = df_test['ner_counts'].apply(create_input_vector_NER)

for tag in ner_labels:
    col_name = f'NER_{tag}'
    df_test[col_name] = df_test['input_vector_ner'].apply(lambda x: x[ner_labels.index(tag)] if tag in ner_labels else 0)
df_test.columns



/var/folders/0q/8tgppr255xx8v40x26xhhmhr0000gn/T/ipykernel_85051/3991742025.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['tot_ner_count'] = df_test['object'].apply(count_named_entities)
/var/folders/0q/8tgppr255xx8v40x26xhhmhr0000gn/T/ipykernel_85051/3991742025.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['ner_counts'] = df_test['object'].apply(count_ner_labels)
/var/folders/0q/8tgppr255xx8v40x26xhhmhr0000gn/T/ipykernel_85051/3991742025.py:5: SettingWithCopyWarning: 
A value is

Index(['json_id', 'claim', 'object', 'compressed_size', 'readability',
       'vader_neg', 'vader_neu', 'vader_pos', 'vader_compound',
       'tot_ner_count', 'ner_counts', 'input_vector_ner', 'NER_CARDINAL',
       'NER_DATE', 'NER_EVENT', 'NER_FAC', 'NER_GPE', 'NER_LANGUAGE',
       'NER_LAW', 'NER_LOC', 'NER_MONEY', 'NER_NORP', 'NER_ORDINAL', 'NER_ORG',
       'NER_PERCENT', 'NER_PERSON', 'NER_PRODUCT', 'NER_QUANTITY', 'NER_TIME',
       'NER_WORK_OF_ART'],
      dtype='object')

In [9]:
from faKy import count_pos, create_input_vector_pos, pos_tags
df_test['pos counts'] = df_test['object'].apply(count_pos)
df_test['input_vector_pos'] = df_test['pos counts'].apply(create_input_vector_pos)
for tag in pos_tags:
    col_name = f'pos_{tag}'
    df_test[col_name] = df_test['input_vector_pos'].apply(lambda x: x[pos_tags.index(tag)] if tag in pos_tags else 0)



/var/folders/0q/8tgppr255xx8v40x26xhhmhr0000gn/T/ipykernel_85051/3395748976.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['pos counts'] = df_test['object'].apply(count_pos)
/var/folders/0q/8tgppr255xx8v40x26xhhmhr0000gn/T/ipykernel_85051/3395748976.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['input_vector_pos'] = df_test['pos counts'].apply(create_input_vector_pos)
/var/folders/0q/8tgppr255xx8v40x26xhhmhr0000gn/T/ipykernel_85051/3395748976.py:6: SettingWithCopyWarning: 
A value

In [10]:

print(df_test['input_vector_ner'].iloc[1])

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 3. 0. 0. 0. 0.]


In [11]:
print(df_test['ner_counts'].iloc[1])

{'PERSON': 3, 'GPE': 1}


In [12]:

text_object = df_test['object'].iloc[1]
print(f'\n{text_object} \n')


Hillary Clinton agrees with John McCain "by voting to give George Bush the benefit of the doubt on Iran." 



In [13]:
import spacy
nlp  = spacy.load('en_core_web_md')

In [14]:
doc = nlp(text_object)

In [15]:
print('\n')
spacy.displacy.render(doc, style= 'ent', jupyter = True)
print('\n')

In [20]:
!pip install svgwrite


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [16]:
from spacy import displacy
from pathlib import Path
import os

In [17]:
options = {"word_spacing": 20, "arrow_spacing": 10, "font_size": 430}

In [18]:
sentence_spans = list(doc.sents)
spacy.displacy.render(sentence_spans, style= 'dep', jupyter = True)
#displacy.serve(sentence_spans, style="dep")


In [19]:
svg = spacy.displacy.render(sentence_spans, style= 'dep', jupyter = False)
with open(os.path.join(os.getcwd(), 'output.svg'), 'w', encoding='utf-8') as f:
    f.write(svg)


In [20]:
print(df_test['pos counts'].iloc[8])

{'VERB': 3, 'PROPN': 5, 'ADJ': 2, 'NOUN': 5, 'CCONJ': 1, 'DET': 2, 'SYM': 1, 'NUM': 2, 'ADP': 1, 'PUNCT': 1}


In [21]:
print(df_test['input_vector_pos'])

0    [1.0, 3.0, 2.0, 1.0, 0.0, 2.0, 0.0, 5.0, 0.0, ...
1    [0.0, 4.0, 0.0, 0.0, 0.0, 2.0, 0.0, 2.0, 0.0, ...
2    [2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 7.0, 0.0, ...
3    [1.0, 2.0, 0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 0.0, ...
4    [3.0, 3.0, 1.0, 1.0, 0.0, 4.0, 0.0, 5.0, 2.0, ...
5    [0.0, 2.0, 1.0, 1.0, 0.0, 1.0, 0.0, 2.0, 0.0, ...
6    [3.0, 4.0, 2.0, 2.0, 0.0, 4.0, 0.0, 5.0, 0.0, ...
7    [1.0, 3.0, 2.0, 0.0, 0.0, 2.0, 0.0, 1.0, 2.0, ...
8    [2.0, 1.0, 0.0, 0.0, 1.0, 2.0, 0.0, 5.0, 2.0, ...
9    [5.0, 3.0, 0.0, 1.0, 0.0, 4.0, 0.0, 7.0, 0.0, ...
Name: input_vector_pos, dtype: object


In [22]:
ner_lst = nlp.pipe_labels['ner']

print(len(ner_lst))
print(ner_lst)

18
['CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART']


In [23]:
text_object = df_test['object'].iloc[1]
doc = nlp(text_object)

In [24]:
sentence_spans = list(doc.sents)
spacy.displacy.render(sentence_spans, style= 'dep', jupyter = True)

In [25]:
svg_ner = spacy.displacy.render(doc, style= 'ent', jupyter = False)
with open(os.path.join(os.getcwd(), 'output_ner.svg'), 'w', encoding='utf-8') as f:
    f.write(svg_ner)